# Hyperparameter Tuning using HyperDrive

Importing Dependencies.

In [ ]:
from azureml.core import Workspace,Experiment
from azureml.core.compute import ComputeTarget, Amlcompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameters_expressions import choice,uniform
from azureml.train.sklearn import SKLearn

## Dataset

Getting Dataset

In [ ]:
ws = Workspace.from_config()
experiment_name = 'heart_faliure'

experiment = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

path = ""

dataset = Dataset.Tabular.from_delimited_files(path)


In [ ]:
# compute target
cpu_compute_name = "cpucompute"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Using existing cluster...')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# Creating an early termination policy. (This is not required if we are using Bayesian sampling.)
early_termination_policy = Bandit_Policy(evaluation_interval = 2,slack_factor = 0.1)

# Creating different params that will be used during training
param_sampling = RandomParameterSampling(
                 "--C": uniform(0.001, 1.0),
                 "--max_iter": choice(50, 100, 200))

# Creating your estimator and hyperdrive config
estimator = SKLearn(source_directory='.',entry_script='train.py',compute_target=cpu_cluster)

hyperdrive_run_config = HyperDriveConfig(estimator = estimator,
                                    hyperparameter_sampling = param_sampling,
                                    policy = early_termination_policy,
                                    primary_metric_name = "Accuracy",
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 4,
                                    max_concurrent_runs = 4)

In [1]:
exp = experiment.submit(config=hyperdrive_run_config, show_output=True)


NameError: name 'experiment' is not defined

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(exp).show()
exp.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = exp.get_best_run_by_primary_metric()
print('Best Run Id: ', best_run.id)
best_run_metrics = best_run.get_metrics()
print(best_run_metrics)

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name = 'heart-failure-prediction-hyperdrive-model',model_path ='outputs/hd-model.joblib', tags = {'Method':'Hyperdrive'}, properties = {'Accuracy': best_run_metrics['Accuracy']})
print(model)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service